In [1]:
import json

import torch
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from transformers import GPT2LMHeadModel

from utils import complete_text

## Tokenizer

The Tokenizer is the object that separate text into tokens.

Tokens are "units" of text: they can be single caracters or multiple words.

There is a fixed set of tokens that is called the vocabulary.

The vocabulary is computed automatically based on the text we used for training the Language model

Here, we compute a vocabulary with 5000 tokens using the text from Wikipedia in Khmer (can take a few minutes).

In [3]:
plain_wikipedia_file = 'plain_wikipedia.txt'
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=5000)
tokenizer.train(files=[plain_wikipedia_file], trainer=trainer)

Below, we use the encode method of the tokenizer to split the text "ប្រាសាទ អង្គរវត្ត" into tokens.

As you can see, tokens are not necessarly words, they can be single characters. If you want to know more about how the vocabulary of token is computed you can have a look here: [https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)

In [4]:
text = "ប្រាសាទ អង្គរវត្ត"
output = tokenizer.encode(text)
print(output.ids)
print(output.tokens)

[3632, 4960, 1138, 3391]
['ប្រាសាទ', ' អង្គ', 'រ', 'វត្ត']


From a sequence of ids, we can build back the sentence using the decode method of the tokenizer.

In [5]:
print(tokenizer.decode(output.ids, skip_special_tokens=False))

ប្រាសាទ  អង្គ រ វត្ត


Important: as you can see, the decoder did not output the exact same sequence that we inputed: 

Input: "ប្រាសាទ អង្គរវត្ត"

Output: "ប្រាសាទ អង្គ រ វត្ត"

The reason is that the Tokenizer we used is not adapted to the Khmer language.

In the future, if you want to develop a languge model for Khmer, you will need to develop you own tokenizer that takes into account the specificity of the language.

## Language Model

Here we are loading a generative language model for khmer that we already trained

In [8]:
initial_model = GPT2LMHeadModel.from_pretrained("gpt2-wikipedia-khmer-no-pretrain")
model = GPT2LMHeadModel.from_pretrained("gpt2-wikipedia-khmer")

You can think of a generative language model (other names include autoregressive language model or causal language model) as a very advanced algorithms for autocomplete:

It takes as input a piece of text and will complete it until a stop token is gneerated or the max length is reached. 

The model is probabilistic: based on the text that we used to train it, it will output the most porbably sequence of token that should follow the input piece of text.

It does NOT use facts from a Knoweldge Base. 

Never trust the output from a generative language model: it generates probable text NOT factual text.

In [13]:
sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"
print("Before training")
complete_text(sentence_to_complete, initial_model, tokenizer)
print("After training")
complete_text(sentence_to_complete, model, tokenizer)

Before training
-5.8727145 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 យក យក យក យក យក យក យក យក យក 朝 ្អូន ្អូន ្អូន
-5.8830476 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 យក យក យក យក យក យក យក យក យក យក យក យក យក
-5.8839407 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪
-5.8843255 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 យក យក យក យក យក យក យក យក យក យក ң ң ң
-5.887452 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 យក យក យក យក យក យក យក យក យក ң ң ң ң
-5.887632 ប្រាសាទ  អង្គ រ វត្ត គ្រូ 別 別 別 別 別 別 別 別 別 別 ˧ ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 ˧ ˧ ˧ ˧ ˧ ˧ ˧ 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 阪 យក យក យក យក យក យក យក យក យក យក យក

In [10]:
sentence_to_complete = "ជំរាបសួរ ខ្ញុំ"
complete_text(sentence_to_complete, model, tokenizer)

-0.3973586 ជំ រាប សួរ  ខ្ញុំ ព្រះករុណ ា  សូម សម ាទ ាន នូវ សិ ក្ខ ាបទ  គឺ ច េត នា ជា ហេតុ វ ៀរ ចាក ហេតុ ជា ទីតាំង នៃ សេចក្តី ប្រ ម ាទ  គឺ ផ ឹក នូវ ទឹក ស្រ វ ឹង  គឺ សុ រ ានិ ង ម េរ ័យ ។
-0.41015318 ជំ រាប សួរ  ខ្ញុំ ព្រះករុណ ា  សូម សម ាទ ាន នូវ សិ ក្ខ ាបទ  គឺ ច េត នា ជា ហេតុ វ ៀរ ចាក ហេតុ ជា ទីតាំង នៃ សេចក្តី ប្រ ម ាទ  គឺ ផ ឹក នូវ ទឹក ស្រ វ ឹង  គឺ សុ រ ានិ ង ម េរ ័យ ។ ស្ វា ធ្យ ាយ ក្នុង
-0.41032267 ជំ រាប សួរ  ខ្ញុំ ព្រះករុណ ា  សូម សម ាទ ាន នូវ សិ ក្ខ ាបទ  គឺ ច េត នា ជា ហេតុ វ ៀរ ចាក ហេតុ ជា ទីតាំង នៃ សេចក្តី ប្រ ម ាទ  គឺ ផ ឹក នូវ ទឹក ស្រ វ ឹង  គឺ សុ រ ានិ ង ម េរ ័យ  ។
-0.41949642 ជំ រាប សួរ  ខ្ញុំ ព្រះករុណ ា  សូម សម ាទ ាន នូវ សិ ក្ខ ាបទ  គឺ ច េត នា ជា ហេតុ វ ៀរ ចាក ហេតុ ជា ទីតាំង នៃ សេចក្តី ប្រ ម ាទ  គឺ ផ ឹក នូវ ទឹក ស្រ វ ឹង  គឺ សុ រ ានិ ង ម េរ ័យ ។ ស្ វា ធ្យ ាយ នូវ
-0.4245352 ជំ រាប សួរ  ខ្ញុំ ព្រះករុណ ា  សូម សម ាទ ាន នូវ សិ ក្ខ ាបទ  គឺ ច េត នា ជា ហេតុ វ ៀរ ចាក ហេតុ ជា ទីតាំង នៃ សេចក្តី ប្រ ម ាទ  គឺ ផ ឹក នូវ ទឹក ស្រ វ ឹង  គឺ សុ រ ានិ ង ម េរ ័យ ។ ស្ វា ធ្យ ាយ  ក្នុង
-0.42643344 ជំ រាប ស

In [11]:
sentence_to_complete = "ហ៊ុន សែន"
complete_text(sentence_to_complete, model, tokenizer)

-0.4562883 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។
-0.4602402 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។
-0.4920465 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ​
-0.51525223 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ពី
-0.5208448 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ពី
-0.52344555 ហ៊ុន  ស ែន ជាន ាយ ក រដ្ឋ មន ្រ្ត ី នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ជាន ាយ ក រ

In [51]:
sentence_to_complete = "ប៊ុនរ៉ានី"
complete_text(sentence_to_complete, model, tokenizer)

-0.51246643 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។
-0.5470465 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -
-0.5515927 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -
-0.5562063 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -
-0.56327087 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ ថ
-0.56541455 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -
-0.5659427 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -
-0.5671319 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ រដ្ឋ
-0.5701202 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ រដ្ឋ
-0.5708964 ប៊ ុន រ៉ ាន ី ន ត ុន រដ្ឋ មន្ត្រី ក្រស ួង យុ ត្តិ ធម៌ នៃ ព្រះរាជា ណាចក្រ កម្ពុជា ។ -


In [54]:
sentence_to_complete = "តើ 1+1 ជាអ្វី?"
complete_text(sentence_to_complete, model, tokenizer)

-0.4089401 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ជា .  ទ ស ក . ,  ខ ុ .  ធ .
-0.41584086 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ជា .  ទ ស ក . ខ ុ .  ធ .
-0.4190509 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ខ ុ .  ធ .
-0.4313643 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ខ ុ .  ធ .
-0.43306038 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ .
-0.43394488 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ជា .  ទ ស ក . ,  ខ ុ .  ជា .  ទ ស ក . ,  ខ ុ .  ធ .
-0.43467146 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ខ ុ .  ធ
-0.4390796 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ជា .  ទ ស ក . ,  ខ ុ .  ជា .  ទ ស ក . ,  ខ ុ .  ធ . , 
-0.44117218 ត ើ  1 + 1  ជា អ្វី ? ខ ុ .  ធ . ) ខ ុ .  ធ . ,  ខ ុ .  ធ . ,  ខ ុ .  ធ . ខ ុ .  ធ .  ជ